# cuSpatial Cheat Sheets sample code

(c) 2020 NVIDIA, Blazing SQL

Distributed under Apache License 2.0

## Imports

In [1]:
import cudf
import cuspatial
import numpy as np
import math
import cupy as cp

# Read data

In [2]:
ring, poly_idx, polygons = cuspatial.io.shapefile.read_polygon_shapefile('../data/gadm36_USA_1.shp')
polygons = polygons.rename(columns={'x': 'longitude', 'y': 'latitude'})
polygons['line'] = cp.random.randint(10,240, len(polygons))
polygons = polygons.query(
        'longitude > -130'
        ' and longitude < 70'
        ' and latitude < 50'
    )

## GeoSpatial functions

#### cuspatial.core.gis.directed_hausdorff_distance

In [3]:
cuspatial.directed_hausdorff_distance(cudf.Series([0,1,0,0]), cudf.Series([0,0,1,2]), [2,2])

,0,1
0,2.350696e-308,2.476726e-308
1,2.602755e-308,0.000000e+00


#### cuspatial.core.gis.haversine_distance

In [5]:
us_center = [39.8333333, -98.585522]
polygons['us_center_lat'] = us_center[0]
polygons['us_center_lon'] = us_center[1]

polygons['distance'] = cuspatial.haversine_distance(
    polygons['latitude']
    , polygons['longitude']
    , polygons['us_center_lat']
    , polygons['us_center_lon']
)
polygons.head()

,longitude,latitude,line,us_center_lat,us_center_lon,distance
0,-87.970001,30.676666,106,39.833333,-98.585522,1178.059263
1,-87.971664,30.676666,124,39.833333,-98.585522,1177.875874
2,-87.971947,30.676390,64,39.833333,-98.585522,1177.844607
3,-87.972221,30.676390,168,39.833333,-98.585522,1177.814323
4,-87.972221,30.675278,161,39.833333,-98.585522,1177.813758


#### cuspatial.core.gis.lonlat_to_cartesian

In [6]:
cuspatial.core.gis.lonlat_to_cartesian(
    us_center[1]
    , us_center[0]
    , polygons['longitude']
    , polygons['latitude']
)


,x,y
0,-963.171193,1017.407449
1,-963.020286,1017.407449
2,-962.996316,1017.438179
3,-962.971396,1017.438179
4,-962.978001,1017.561732
...,...,...
1298509,984.061842,-129.321914
1298510,982.117386,-129.358366
1298511,979.216187,-129.398632
1298512,970.456382,-129.491880


#### cuspatial.core.gis.point_in_polygon

In [7]:
poi = [
    [47.620422, -122.349358]     ### Space Needle
    , [43.8802654, -103.4588021] ### Mt Rushmore
    , [40.689247, -74.044502]    ### Statue of Liberty
]

poi = list(map(list, zip(*poi))) # transpose
poi

cuspatial.point_in_polygon(
      poi[0]
    , poi[1]
    , cudf.Series([0,1,2], index=['WA','SD','NY'])
    , [0,3,8]
    , [50,46,46,47,45,39,40,44,41,38,39,42]
    , [-124,-123,-120,-121,-104,-105,-100,-101,-75,-76,-70,-69]
)

,WA,SD,NY
0,True,False,False
1,False,True,False
2,False,False,True


#### cuspatial.core.gis.polygon_bounding_boxes

In [9]:
result = cuspatial.polygon_bounding_boxes(
    [0, -8, 6.0],                             # test_points_x
    [0, -8, 6.0],                             # test_points_y
    cudf.Series([0, 1], index=['nyc', 'hudson river']), # poly_offsets
    [0, 3],                                   # ring_offsets
    [-10, 5, 5, -10, 0, 10, 10, 0],           # poly_points_x
    [-10, -10, 5, 5, 0, 0, 10, 10]           # poly_points_y
)

result

TypeError: polygon_bounding_boxes() takes 4 positional arguments but 6 were given

#### cuspatial.core.gis.polyline_bounding_boxes

In [10]:
cuspatial.polyline_bounding_boxes(
    poly_idx
    , polygons['longitude']
    , polygons['latitude']
    , 0.01
)

RuntimeError: transform: failed to synchronize: cudaErrorIllegalAddress: an illegal memory access was encountered

#### cuspatial.core.indexing.quadtree_on_points
CubicSpline
CubicSpline.__init__
CubicSpline.__call__

#### cuspatial.core.spatial_window.points_in_spatial_window

#### cuspatial.core.spatial_join.join_quadtree_and_bounding_boxes
#### cuspatial.core.spatial_join.quadtree_point_in_polygon
#### cuspatial.core.spatial_join.quadtree_point_to_nearest_polyline

#### cuspatial.core.trajectory.derive_trajectories
#### cuspatial.core.trajectory.trajectory_bounding_boxes
#### cuspatial.core.trajectory.trajectory_distances_and_speeds

#### cuspatial.io.shapefile.read_polygon_shapefile